In [1]:
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import os
import sqlalchemy as sq

load_dotenv(find_dotenv())
host = os.getenv("DB_HOST")
db_pass = os.getenv("DB_PASSWORD")
db_user = os.getenv("DB_USERNAME")
db_database = os.getenv("DATABASE")

In [3]:
# creating sql instance
engine = sq.create_engine(
    f'mysql+pymysql://{db_user}:{db_pass}@{host}:3306/{db_database}')

In [6]:
df=pd.read_sql('todayshare',engine)

In [16]:
df.drop_duplicates(subset=['Date','Traded Companies'],inplace=True)
df.drop(columns=['index'],inplace=True)

In [20]:
columns_x = ['Amount', 'Closing Price', 'Max Price',
             'Min Price', 'Previous Closing', 'Traded Shares']

In [21]:
def numerical_data_maker(df, columns):
    for i in columns:
        if type(list(df[i])[0]) == float or int:
            df[i] = pd.to_numeric(df[i])
        else:
            df[i] = pd.to_numeric(df[i].str.replace(',', ''))
    return df

In [23]:
df=numerical_data_maker(df,columns_x)

In [27]:
df['Date'] = pd.to_datetime(df['Date'])

In [29]:
df.sort_values('Date',ascending=False,inplace=True)

In [33]:
df.columns

Index(['Traded Companies', 'No. Of Transaction', 'Max Price', 'Min Price',
       'Closing Price', 'Traded Shares', 'Amount', 'Previous Closing',
       'Difference Rs.', 'Date'],
      dtype='object')

In [88]:
def vwap(df,vol_col_name,price_col_name):
    traded_amount = df[vol_col_name].values
    closing_price = df[price_col_name].values
#     return df.assign(vwap=(closing_price * traded_amount).cumsum() / traded_amount.cumsum())

df_adbl = df[df['Traded Companies']=='Agriculture Development Bank Limited']
adbl_vwap = vwap_values=vwap(df_adbl,'Amount','Closing Price')
df_adbl=df_adbl[df_adbl['Closing Price']!=0]

In [89]:
v = df_adbl['Amount'].values
tp = (df_adbl['Min Price'] + df_adbl['Closing Price'] + df_adbl['Max Price']).div(3).values

In [90]:
vwap=(tp * v).cumsum() / v.cumsum()

In [91]:
vwap

array([565.56666667, 567.60498059, 569.12178638, ..., 513.31879626,
       513.29866231, 513.28444802])

In [92]:
vwap_in_list = []
for i in range(len(tp)):
    limit = 7+i
    vwap_in_list += [np.sum(tp[:limit]*v[:limit])/np.sum(v[:limit])]

In [93]:
df_adbl['vwap']=vwap_in_list

In [94]:
np.sum(tp[:limit]*v[:limit])/np.sum(v[:limit])

513.284448018968

In [95]:
import plotly.graph_objects as go
import numpy as np

N = 1000
t = np.linspace(0, 10, 100)
y = np.sin(t)

fig = go.Figure(data=go.Scatter(y=df_adbl['vwap'],x=df_adbl['Date'] , mode='markers',name='VWAP'))
fig.add_trace(go.Scatter(y=df_adbl['Closing Price'],x=df_adbl['Date'],mode='lines',name='Closing Price'))
fig.add_trace(go.Scatter(y=df_adbl['Closing Price']-df_adbl['vwap'],x=df_adbl['Date'],mode='lines',name='Diff'))

fig.show()

In [96]:
df_adbl['diff']=df['Closing Price']-df_adbl['vwap']

In [97]:
df_adbl[['vwap','diff','Closing Price']].describe()

,vwap,diff,Closing Price
count,2422.000000,2422.000000,2422.000000
mean,503.829611,-117.029755,386.799855
std,21.447336,175.354496,172.624491
min,470.070431,-413.504544,101.000000
25%,481.081854,-260.782546,257.250000
50%,514.030502,-80.679260,418.000000
75%,518.091390,-32.636002,473.000000
max,590.213130,576.696492,1082.000000


In [102]:
# percentage of time that vwap is lower than the closing price
per_vwap_more_than_cp=df_adbl[df_adbl['Closing Price']>df_adbl['vwap']].shape[0]/df_adbl.shape[0]
# percentage of time that vwap is more than the closing price
per_vwap_less_than_cp=df_adbl[df_adbl['Closing Price']<df_adbl['vwap']].shape[0]/df_adbl.shape[0]

In [111]:
df_adbl['vwap_diff']=(df_adbl['diff']*100/df_adbl['vwap'])

In [112]:
df_adbl['vwap_diff'].describe()

count    2422.000000
mean      -22.974642
std        34.416543
min       -80.369464
25%       -50.395866
50%       -15.995696
75%        -6.674598
max       118.102058
Name: vwap_diff, dtype: float64

In [115]:

fig = go.Figure(data=[go.Histogram(y=df_adbl['vwap_diff'],histnorm='probability')])

fig.show()